In [ ]:
import pandas as pd
import numpy as np

In [ ]:
FPS = 30

In [ ]:
dfraw = pd.read_csv('../logs/log_lanes2.csv', header=None)
dfraw.columns = ['frame', 'lane', 'objectId', 'objectType', 'secMark', 'xLeft', 'xRight', 'yTop', 'yBottom', 'lat', 'lon', 'speed', 'heading', 'elevation'] 
dfraw.speed = dfraw.speed.replace('None', np.nan).astype(float)
print (dfraw.shape)
print (dfraw.columns)
dfraw.head()

In [ ]:
df = dfraw.copy()
#df.loc[df.lane==4, 'lane'] = 3
df = df.groupby(['objectId', 'lane']).agg({
    'secMark' : 'count',
    'frame' : [np.min, np.max],
    'speed' : np.nanmean,
})
df.columns = ['nr_frames', 'frame_start', 'frame_end', 'speed_mean']
df = df.reset_index()
print (df.shape)
df = df[df.nr_frames >= 0.5 * FPS] # Must be detected for at least X seconds
df = df.sort_values('frame_end')
print (df.shape)
df.head(10)

In [ ]:
df = df.groupby(['lane', 'nr_frames', 'frame_start', 'frame_end', 'speed_mean']).agg({'objectId' : 'min'}).reset_index()
df = df[['objectId', 'lane', 'nr_frames', 'frame_start', 'frame_end', 'speed_mean']]
df = df.sort_values('frame_end')
print (df.shape)
df.head(20)

In [ ]:
laneObjects = {}
for lane in sorted(list(df.lane.unique())):
    laneObjects[lane] = df.objectId[df.lane == lane].unique()
    print ('{} objects found at lane {}'.format(len(laneObjects[lane]), lane))

In [ ]:
OLD_VERSION = False
if OLD_VERSION:
    df_noNeed = df[~df.objectId.isin(df.objectId[df.lane==2])]
    print (df_noNeed.shape)
    df_noNeed = df_noNeed.groupby('objectId').agg({
        'frame_start' : 'min',
        'frame_end' : 'max',
        'nr_frames' : 'sum',
    }).reset_index()
    print (len(df_noNeed.objectId.unique()), df_noNeed.shape)
else:
    df_noNeed = df[df.lane == 4]
    print (df_noNeed.shape)
    df_noNeed = df_noNeed[~df_noNeed.objectId.isin(laneObjects[2])]
    print (df_noNeed.shape)
    df_noNeed = df_noNeed[df_noNeed.objectId.isin(laneObjects[3])]
    print (df_noNeed.shape)
df_noNeed.head(20)    

In [ ]:
print (len (df_noNeed))
for i in range (7):
    print(i,  len(df_noNeed[df_noNeed.nr_frames > 30*i]))

In [ ]:
if False:
    df_noNeed = df_noNeed[df_noNeed.nr_frames > FPS * 2] # Must be detected for at least X seconds
    print (len(df_noNeed.objectId.unique()), df_noNeed.shape)
    df_noNeed.head()

In [ ]:
df_moveover = df[df.lane == 2]
print (df_moveover.shape)
df_moveover = df_moveover[df.objectId.isin(laneObjects[3])]
print (df_moveover.shape)
print (len(df_moveover.objectId.unique()))
df_moveover.head()

In [ ]:
# Old version
if OLD_VERSION:
    tmp = pd.concat([df_noNeed, df_moveover], ignore_index=True).objectId.unique()
    print (len(tmp))
    df_noMoveOver = df[~df.objectId.isin(tmp)]
    print (df_noMoveOver.shape)
    df_noMoveOver = df_noMoveOver[df_noMoveOver.objectId.isin(df.loc[df.lane==1, 'objectId'])]
    print (df_noMoveOver.shape)
    df_noMoveOver.head()
else:
    df_noMoveOver = df[df.lane == 2]
    print (df_noMoveOver.shape)
    df_noMoveOver = df_noMoveOver[df_noMoveOver.objectId.isin(laneObjects[1])]
    print (df_noMoveOver.shape)
    #df_noMoveOver = df_noMoveOver[~df_noMoveOver.objectId.isin(laneObjects[4])]
    #print (df_noMoveOver.shape)
    
    df_noMoveOver.head()    

In [ ]:
if OLD_VERSION:
    tmp = df_noMoveOver[['objectId', 'lane', 'frame_start', 'frame_end', 'speed_mean']]
    dfSpeed = tmp[tmp.lane == 2]
    print (dfSpeed.shape)
    dfSpeed = dfSpeed.merge(tmp[tmp.lane == 1], on='objectId')
else:
    dfSpeed = df_noMoveOver.merge(df[df.lane == 1], on = 'objectId')

print (dfSpeed.shape)    
dfSpeed = dfSpeed[['objectId', 'frame_start_x', 'frame_start_y', 'speed_mean_x', 'speed_mean_y']]
dfSpeed.columns = ['objectId', 'frame_start', 'frame_end', 'speed_before', 'speed_after']
dfSpeed['slowed'] = dfSpeed.speed_before * 0.95 > dfSpeed.speed_after
dfSpeed['action'] = 'noSlow'
dfSpeed.loc[dfSpeed.slowed, 'action'] = 'slow'
dfSpeed = dfSpeed.sort_values('frame_end')
dfSpeed.head()


In [ ]:
dfSpeed.action.value_counts()

In [ ]:
dfres = df_noNeed[['objectId', 'frame_start', 'frame_end']].copy()
dfres['action'] = 'noNeed'
df_moveover['action'] = 'moveOver'
dfres = pd.concat([dfres, df_moveover[['objectId', 'frame_start', 'frame_end', 'action']]], ignore_index=True)
dfres = pd.concat([dfres, dfSpeed[['objectId', 'frame_start', 'frame_end', 'action']]], ignore_index=True)
dfres = dfres.sort_values('frame_end')
dfres

In [ ]:
dfres.action.value_counts()

In [ ]:
dfres.set_index('objectId').to_csv('../CBTV44/actions.left.csv')

In [ ]:
dfres.action[dfres.frame_end < 5*60*FPS].value_counts()

In [ ]:
dfres.head()

# Fast video generation

In [ ]:
import glob
import os
import sys
import cv2
import numpy as np
import matplotlib.pyplot as plt

from time import time

In [ ]:
TEXT_SPACE_BOTTOM = 200
OUTPUT_FRAMES_PATH = '../CBTV44/frames/'
INPUT_FRAMES_PATH = '../frames/'
VIDEO_X = 800
VIDEO_Y = 450
df = pd.read_csv('../CBTV44/actions.left.csv').sort_values('frame_end')
df.head(20)

In [ ]:
noNeed = 0
moveOver = 0
slow = 0
noSlow = 0
def displayImageCounter(img_y, img_x, nr_frame, return_int = True):
    global noNeed
    global moveOver
    global slow
    global noSlow
    
    im = np.zeros((img_y, img_x,3))
    font = cv2.FONT_HERSHEY_PLAIN
    
    tmp = df[df.frame_end == nr_frame]
    if len(tmp) > 0:
        noNeed += len(tmp[tmp.action == 'noNeed'])
        moveOver += len(tmp[tmp.action == 'moveOver'])
        slow += len(tmp[tmp.action == 'slow'])
        noSlow += len(tmp[tmp.action == 'noSlow'])
    
    line_width = 45
    color = (0, 1, 0.5)
    cv2.putText(im, 'No need to move over: {}'.format(noNeed),    (10,line_width), font, 2, color, 2, cv2.LINE_AA)
    cv2.putText(im, 'Moved over: {}'.format(moveOver), (10, 2 * line_width), font, 2, color, 2, cv2.LINE_AA)
    cv2.putText(im, "Didn't move over, speed reduced: {}".format(slow),     (10, 3 * line_width), font, 2, color, 2, cv2.LINE_AA)
    cv2.putText(im, "Didn't move over, speed not reduced: {}".format(noSlow),     (10, 4 * line_width), font, 2, color, 2, cv2.LINE_AA)
    if return_int:
        im = (255 * im).astype(np.byte)
    return im

tmp = displayImageCounter(TEXT_SPACE_BOTTOM, 850, 19, return_int = False)
plt.imshow(tmp)

In [ ]:
filenames = glob.glob(os.path.join(INPUT_FRAMES_PATH, '*.jpg'))
filenames.sort()


In [ ]:
t = time()
for i, filename in enumerate(filenames):
    image = cv2.imread(filename)
    image = image[:VIDEO_Y, :VIDEO_X, :]
    final_image = np.zeros((image.shape[0] + TEXT_SPACE_BOTTOM, image.shape[1], 3),dtype=np.uint8)

    final_image[:image.shape[0], :VIDEO_X, :] = image
    tmp = displayImageCounter(TEXT_SPACE_BOTTOM, image.shape[1], i)
    final_image[VIDEO_Y:, :, : ] = tmp 
    
    #final_image = cv2.cvtColor(final_image, cv2.COLOR_RGB2BGR)

    cv2.imwrite(os.path.join(OUTPUT_FRAMES_PATH, 'im_{}.jpg'.format(str(i).zfill(5))), final_image)
    if i%FPS == 0:
        t2 = time() - t
        minutes = int (i/FPS / 60)
        seconds = int (i/FPS - (minutes * 60))
        sys.stdout.write('{} frames ({}:{} min) done in {:.1f} seconds ({:.2f} frames/sec)    \r'.format(
            i, str(minutes).zfill(2), str(seconds).zfill(2), t2, i/t2))                   
 

In [ ]:
from helper import generate_video
generate_video(fps = 60, inputPath = OUTPUT_FRAMES_PATH, outputFile='output_CBTV44.avi')